This script converts combined labels to a dataset that huggingface can use

In [1]:
import sys
sys.path.append('../../../../lib/')

In [2]:
from analysis import *
import pandas as pd
import json

In [3]:
# Read Combinedlabels.csv
df = pd.read_csv("Combinedlabels.csv")

In [4]:
print(df.head())

   tweet_id  Gabe  Guanyi  Mom  Sum  \
0         0     0       0    0    0   
1         1     0       0    0    0   
2         2     0       0    0    0   
3         3     1       1    1    3   
4         4     0       0    0    0   

                                          tweet_text  \
0  RT {{MENTION}}: Starting in one hour. Raising ...   
1  $102.82 this week! This is how I now make a pa...   
2  OPEN today 10 am until 1 pm\n419 CCPkwy Cape C...   
3  A superb 18th century European carved ivory di...   
4                                 Driving aroundd :P   

                                    user_description  
0  Can't start a fire... Can't start a fire witho...  
1                     Follow me and you'll find out!  
2  We are 3rd generation Brocante-Antique Import ...  
3  John Nicholson's Auctioneers are the largest a...  
4  Im 13 years young\nim in love with the most am...  


In [5]:
# Read in ocr json
with open('text_from_images/ids_to_text.json') as f:
	ocr_json = json.load(f)

In [6]:
texts = []
labels = []
max_length = 0
for index, row in df.iterrows():
		label = row['Sum']
		if label >= 2:
			labels.append(1)
		else:
			labels.append(0)
		try:
			text = str(row["tweet_text"]) + " [sep] " + str(row["user_description"] + " [sep] " + ocr_json[str(row["tweet_id"])])
		except:
			text = str(row["tweet_text"]) + " [sep] " + str(row["user_description"])
		text = text.replace('"', '""')
		if len(text) > max_length:
			max_length = len(text)
		texts.append(text)
print(str(max_length))

2128


In [7]:
good_tweets = {}
bad_tweets = {}
good_tweets["tweets"] = []
bad_tweets["tweets"] = []

for tweet in range(len(texts)):
	out = {}
	out["text"] = texts[tweet]
	if labels[tweet] == 0:
		good_tweets["tweets"].append(out)
	else:
		bad_tweets["tweets"].append(out)

In [8]:
# Write to good_tweets.json
with open('good_tweets.json', 'w') as outfile:
	json.dump(good_tweets, outfile)

with open('bad_tweets.json', 'w') as outfile:
	json.dump(bad_tweets, outfile)